train, test 데이터의 'latitude', 'longitude'열, jeju_financial_life_data의 'X_AXIS', 'Y_AXIS'열을 지오코딩 프로그램을 사용하여 주소로 변환하였다. 
변환한 주소를 전처리 하여 동 기준으로 두 데이터를 합쳐주었다. 그 뒤 파생변수를 생성하였다. data_address.csv는 data의 주소가, 
life_address.csv는 jeju_finantial_life_data의 주소가 담겨있다.

In [35]:
from geopy.geocoders import Nominatim
import pandas as pd
import ssl
import certifi
import ssl; 
ssl._create_default_https_context = ssl._create_stdlib_context



In [36]:
# nominatim이 아닌 법정 행정구역 정보 제공하는 API사용해야함.

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(timeout=None, user_agent="https")
    address = geolocoder.reverse(lat_lng_str)
    print(address)
    return address

In [37]:
data = pd.concat(
	[pd.read_csv('./datas/train.csv'), pd.read_csv('./datas/test.csv')],
	axis=0
)


In [38]:
#data의 위도 경도를 합쳐서 하나의 문자열로 만들어준다.
data['location'] = data['latitude'].astype(str) + ', ' + data['longitude'].astype(str)


In [39]:
data['location'].head()

0              33.4899, 126.49373
1             33.48944, 126.48508
2             33.48181, 126.47352
3             33.50577, 126.49252
4    33.255790000000005, 126.4126
Name: location, dtype: object

In [42]:
#위도 경도를 합친 문자열을 geocoding_reverse 함수에 넣어준다.
data['address'] = data['location'].apply(geocoding_reverse)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=33.4899&lon=126.49373&format=json&addressdetails=1 (Caused by SSLError(SSLError(5, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1007)')))

In [ ]:
#주소를 'dong', 'si' 로 나누어서 저장한다.
data['dong'] = data['address'].apply(lambda x: x.raw['address']['suburb'] if 'suburb' in x.raw['address'] else x.raw['address']['town'])
data['si'] = data['address'].apply(lambda x: x.raw['address']['city'] if 'city' in x.raw['address'] else x.raw['address']['county'])

KeyError: 'address'

In [41]:
life_ = pd.read_csv('./datas/jeju_financial_life_data.csv')